# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql('select * from disaster_msgs',engine)

In [4]:
X = df['message']
Y = df.iloc[:, 4:]

In [5]:
Y.columns.tolist()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [6]:
import nltk
import re
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
for message in X[:5]:
    print(message)
    print(tokenize(message))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']
Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']
Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'of', 'haiti', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier((AdaBoostClassifier())))
    #('clf', MultiOutputClassifier(RandomForestClassifier()))
    #('clf', MultiOutputClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
Y_pred = pipeline.predict(X_test)

In [18]:
from sklearn.metrics import classification_report
def test_model(Y_test, Y_pred):
    
      for index, column in enumerate(Y_test):
        print(column, classification_report(Y_test[column], Y_pred[:, index]))

In [19]:
test_model(Y_test, Y_pred)

related              precision    recall  f1-score   support

          0       0.59      0.22      0.32      1566
          1       0.79      0.95      0.86      4937
          2       0.36      0.10      0.15        51

avg / total       0.74      0.77      0.73      6554

request              precision    recall  f1-score   support

          0       0.91      0.96      0.94      5441
          1       0.76      0.54      0.63      1113

avg / total       0.89      0.89      0.89      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.08      0.04      0.05        28

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.75      0.87      0.81      3824
          1       0.77      0.60      0.68      2730

avg / total       0.76      0.76      0.75      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


weather_related              precision    recall  f1-score   support

          0       0.88      0.96      0.92      4752
          1       0.86      0.66      0.75      1802

avg / total       0.88      0.88      0.87      6554

floods              precision    recall  f1-score   support

          0       0.96      0.99      0.98      6039
          1       0.87      0.51      0.65       515

avg / total       0.95      0.96      0.95      6554

storm              precision    recall  f1-score   support

          0       0.95      0.98      0.97      5937
          1       0.76      0.49      0.60       617

avg / total       0.93      0.94      0.93      6554

fire              precision    recall  f1-score   support

          0       0.99      1.00      0.99      6484
          1       0.57      0.17      0.26        70

avg / total       0.99      0.99      0.99      6554

earthquake              precision    recall  f1-score   support

          0       0.98      0.99      0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0b039b3510>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [21]:
from sklearn.model_selection import GridSearchCV
# specify parameters for grid search 

parameters = {
    'clf__estimator__n_estimators' : [50, 100]
}
cv = GridSearchCV(pipeline, param_grid=parameters)

In [22]:
cv.fit(X_train, Y_train)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
cv.best_params_

{'clf__estimator__n_estimators': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
Y_pred = cv.predict(X_test)
test_model(Y_test, Y_pred)

related              precision    recall  f1-score   support

          0       0.59      0.22      0.32      1566
          1       0.79      0.95      0.86      4937
          2       0.38      0.10      0.16        51

avg / total       0.74      0.77      0.73      6554

request              precision    recall  f1-score   support

          0       0.91      0.96      0.94      5441
          1       0.76      0.54      0.63      1113

avg / total       0.89      0.89      0.89      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.08      0.04      0.05        28

avg / total       0.99      0.99      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.75      0.87      0.81      3824
          1       0.77      0.60      0.68      2730

avg / total       0.76      0.76      0.75      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


other_infrastructure              precision    recall  f1-score   support

          0       0.96      0.99      0.98      6285
          1       0.31      0.08      0.13       269

avg / total       0.93      0.95      0.94      6554

weather_related              precision    recall  f1-score   support

          0       0.88      0.96      0.92      4752
          1       0.86      0.66      0.75      1802

avg / total       0.88      0.88      0.87      6554

floods              precision    recall  f1-score   support

          0       0.96      0.99      0.98      6039
          1       0.87      0.51      0.65       515

avg / total       0.95      0.96      0.95      6554

storm              precision    recall  f1-score   support

          0       0.95      0.98      0.97      5937
          1       0.76      0.49      0.60       617

avg / total       0.93      0.94      0.93      6554

fire              precision    recall  f1-score   support

          0       0.99      1.0

In [25]:
print("Accuracy")
print((Y_pred == Y_test).mean())
for i in range(35):
    
    print("Precision, Recall, F1 Score for {}".format(Y_test.columns[i]))
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

Accuracy
related                   0.768996
request                   0.892890
offer                     0.994202
aid_related               0.758926
medical_help              0.927983
medical_products          0.958804
search_and_rescue         0.975740
security                  0.981233
military                  0.972994
child_alone               1.000000
water                     0.961855
food                      0.946750
shelter                   0.946597
clothing                  0.987794
money                     0.979707
missing_people            0.988557
refugees                  0.967806
death                     0.967348
other_aid                 0.874428
infrastructure_related    0.937138
transport                 0.961855
buildings                 0.959109
electricity               0.980623
tools                     0.991913
hospitals                 0.987946
shops                     0.996033
aid_centers               0.988251
other_infrastructure      0.954684
weather_rel

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.96      0.99      0.98      6039
          1       0.87      0.51      0.65       515

avg / total       0.95      0.96      0.95      6554

Precision, Recall, F1 Score for storm
             precision    recall  f1-score   support

          0       0.95      0.98      0.97      5937
          1       0.76      0.49      0.60       617

avg / total       0.93      0.94      0.93      6554

Precision, Recall, F1 Score for fire
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6484
          1       0.57      0.17      0.26        70

avg / total       0.99      0.99      0.99      6554

Precision, Recall, F1 Score for earthquake
             precision    recall  f1-score   support

          0       0.98      0.99      0.98      5930
          1       0.86      0.78      0.82       624

avg / total       0.97      0.97      0.97      6554

Precision, Recall, F1 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [20]:
import pickle
filename = 'Final_model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.